In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU
from tensorflow.keras.models import Sequential

In [2]:
from tensorflow.keras.utils import to_categorical

In [3]:
with open("tolstoy.txt", 'r') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower()
        if len(line) == 0:
            continue
        lines.append(line)
txt = " ".join(lines)

для ускорения сократим объём текста

In [4]:
print(txt[:500],'\n')
print(f'Длинна первоначального текста: {len(txt)} символов')
text = txt[:len(txt)//4]
print(f'Длинна сокращенного текста:{len(text)} символов')

спасибо, что скачали книгу в бесплатной электронной библиотеке royallib.com: https://royallib.com все книги автора: https://royallib.com/author/tolstoy_lev.html эта же книга в других форматах: https://royallib.com/book/tolstoy_lev/voyna_i_mir_tom_1_i_2.html приятного чтения! лев толстой война и мир тома первый и второй в. шкловский «война и мир» льва толстого замысел в 1855 году появилось объявление об издании «полярной звезды». на обложке книги в круге восходящего солнца были изображены пять по 

Длинна первоначального текста: 1623677 символов
Длинна сокращенного текста:405919 символов


In [5]:
vocab = sorted(set(text))
vocab_size = len(vocab)
vocab[:15]

[' ', '!', '#', '&', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3']

## Вариант рассмотренный в классе

основанный на предсказании всей последовательности +1 символ

In [6]:
seq_length = 100

In [7]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

In [8]:
examples_per_epoch = len(text)//(seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [9]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'спасибо, что скачали книгу в бесплатной электронной библиотеке royallib.com: https://royallib.com все'
' книги автора: https://royallib.com/author/tolstoy_lev.html эта же книга в других форматах: https://r'
'oyallib.com/book/tolstoy_lev/voyna_i_mir_tom_1_i_2.html приятного чтения! лев толстой война и мир том'
'а первый и второй в.\xa0шкловский «война и мир» льва толстого замысел в 1855 году появилось объявление о'
'б издании «полярной звезды». на обложке книги в круге восходящего солнца были изображены пять портрет'


In [10]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [11]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'спасибо, что скачали книгу в бесплатной электронной библиотеке royallib.com: https://royallib.com вс'
Target data: 'пасибо, что скачали книгу в бесплатной электронной библиотеке royallib.com: https://royallib.com все'


In [12]:
batch_size = 64
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int32, name=None), TensorSpec(shape=(64, 100), dtype=tf.int32, name=None))>

In [13]:
embedding_dim = 128
rnn_units = 512

In [14]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ]
)

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         12160     
                                                                 
 lstm (LSTM)                 (None, None, 512)         1312768   
                                                                 
 dense (Dense)               (None, None, 95)          48735     
                                                                 
Total params: 1,373,663
Trainable params: 1,373,663
Non-trainable params: 0
_________________________________________________________________


In [16]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [17]:
model.compile(optimizer='adam', loss=loss)

In [18]:
def generate_text(model, start_string):

    num_generate = 100
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []

    temperature = 0.5

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [19]:
def myfit(dataset, model, epochs, start_string):

    for iteration in range(epochs):
        print("=" * 50)
        print("iteration #: %d" % (iteration))
        model.fit(dataset, epochs=1)
        if (iteration+1)%10==0:
            print("Start_string: %s" % (start_string))
            print(generate_text(model, start_string))

In [20]:
myfit(dataset, model, 40, 'выхожу один я на дорогу')

iteration #: 0
62/62 [==============================] - 60s 925ms/step - loss: 3.4954
iteration #: 1
62/62 [==============================] - 58s 928ms/step - loss: 2.9205
iteration #: 2
62/62 [==============================] - 57s 916ms/step - loss: 2.6236
iteration #: 3
62/62 [==============================] - 59s 937ms/step - loss: 2.5039
iteration #: 4
62/62 [==============================] - 57s 920ms/step - loss: 2.4052
iteration #: 5
62/62 [==============================] - 58s 924ms/step - loss: 2.3119
iteration #: 6
62/62 [==============================] - 57s 911ms/step - loss: 2.2276
iteration #: 7
62/62 [==============================] - 60s 968ms/step - loss: 2.1529
iteration #: 8
62/62 [==============================] - 61s 975ms/step - loss: 2.0893
iteration #: 9
62/62 [==============================] - 60s 965ms/step - loss: 2.0305
Start_string: выхожу один я на дорогу
выхожу один я на дорогу осто ви порыва сть грогей и вели пора, уй по о в вовы нали мазана ко вала стот

## Вариант 2

основанный на предсказании одного (следующего) симвлола 
и в качестве эмбеддинга символов простое OHE кодирование 

In [22]:
seq_length, step = 20, 1
train_seq, label_chars = [], []

for i in range(0, len(text) - seq_length, step):
    train_seq.append(text[i: i + seq_length])
    label_chars.append(text[i + seq_length])

In [23]:
train_seq[:2], label_chars[:2]

(['спасибо, что скачали', 'пасибо, что скачали '], [' ', 'к'])

In [24]:
train = []
for seq in train_seq:
    seq_hot = []
    for char in seq:
        char_hot = to_categorical(char2idx[char], num_classes=len(vocab))
        seq_hot.append(char_hot)  
    train.append(seq_hot)
train = tf.convert_to_tensor(train)   
train.shape

TensorShape([405899, 20, 95])

In [25]:
label = []
for char in label_chars:
    char_hot = to_categorical(char2idx[char], num_classes=len(vocab))    
    label.append(char_hot)
label = tf.convert_to_tensor(label)
label.shape

TensorShape([405899, 95])

In [26]:
model_1 = tf.keras.Sequential(
    [
        
        tf.keras.layers.LSTM(rnn_units, input_shape=(seq_length, vocab_size),return_sequences=False),
        tf.keras.layers.Dense(vocab_size, activation='softmax')
    ]
)

In [27]:
model_1.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [28]:
len_generation = 100

In [29]:
def generate_text_1(model, start_string):
    test_chars = start_string[-20:]
    text_generated = []
    
    
    for i in range(len_generation):
        X_test = []
        seq_hot = []

        for char in test_chars:


            char_hot = to_categorical(char2idx[char], num_classes=len(vocab))
            seq_hot.append(char_hot) 
            
        X_test.append(seq_hot)
        X_test = tf.convert_to_tensor(X_test)        
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = idx2char[np.argmax(pred)]
        text_generated.append(y_pred)

        test_chars = test_chars[1:] + y_pred

    return (start_string + ''.join(text_generated))

In [30]:
def myfit_1(train, label, model, epochs, start_string):

    for iteration in range(epochs):
        print("=" * 50)
        print("iteration #: %d" % (iteration))
        model.fit(x=train, y=label, batch_size=batch_size, epochs=1)
        if (iteration+1)%2==0:
            print("Start_string: %s" % (start_string))
            print(generate_text_1(model, start_string))
    return model

In [32]:
model =  myfit_1(train, label, model_1, 6, 'выхожу один я на дорогу')

iteration #: 0
6343/6343 [==============================] - 1136s 177ms/step - loss: 2.2260
iteration #: 1
6343/6343 [==============================] - 1134s 179ms/step - loss: 1.7570
Start_string: выхожу один я на дорогу
выхожу один я на дорогу и приехала в гостиную и подошла к стару с своим и подошла к князю андрей с том и приемания в гостин
iteration #: 2
6343/6343 [==============================] - 1140s 180ms/step - loss: 1.5860
iteration #: 3
6343/6343 [==============================] - 1275s 201ms/step - loss: 1.4763
Start_string: выхожу один я на дорогу
выхожу один я на дорогу от него и отвернулась к приказанием в ответенности полкового командира приступила на коленами, подо
iteration #: 4
6343/6343 [==============================] - 1490s 235ms/step - loss: 1.3867
iteration #: 5
6343/6343 [==============================] - 1197s 189ms/step - loss: 1.3151
Start_string: выхожу один я на дорогу
выхожу один я на дорогу от него никого не сказал он с тогому командику, который он все

## Выводы:


1. Каждая архитектура имеет свои плюсы и минусы, и большинство задач может быть решено различными подходами и различными архитектурами. И не всегда сложная архитектура является лучшим решением.

2. На мой взгляд второй вариант показал более интересный результат.